In [11]:
import torch
import torchvision
import torchvision.transforms as transforms

In [12]:
# Asignamos cuda o CPU una variable
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [13]:
transform = transforms.ToTensor()

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=5, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=5, shuffle=True, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [15]:
print(trainset[0][0])

tensor([[[0.2314, 0.1686, 0.1961,  ..., 0.6196, 0.5961, 0.5804],
         [0.0627, 0.0000, 0.0706,  ..., 0.4824, 0.4667, 0.4784],
         [0.0980, 0.0627, 0.1922,  ..., 0.4627, 0.4706, 0.4275],
         ...,
         [0.8157, 0.7882, 0.7765,  ..., 0.6275, 0.2196, 0.2078],
         [0.7059, 0.6784, 0.7294,  ..., 0.7216, 0.3804, 0.3255],
         [0.6941, 0.6588, 0.7020,  ..., 0.8471, 0.5922, 0.4824]],

        [[0.2431, 0.1804, 0.1882,  ..., 0.5176, 0.4902, 0.4863],
         [0.0784, 0.0000, 0.0314,  ..., 0.3451, 0.3255, 0.3412],
         [0.0941, 0.0275, 0.1059,  ..., 0.3294, 0.3294, 0.2863],
         ...,
         [0.6667, 0.6000, 0.6314,  ..., 0.5216, 0.1216, 0.1333],
         [0.5451, 0.4824, 0.5647,  ..., 0.5804, 0.2431, 0.2078],
         [0.5647, 0.5059, 0.5569,  ..., 0.7216, 0.4627, 0.3608]],

        [[0.2471, 0.1765, 0.1686,  ..., 0.4235, 0.4000, 0.4039],
         [0.0784, 0.0000, 0.0000,  ..., 0.2157, 0.1961, 0.2235],
         [0.0824, 0.0000, 0.0314,  ..., 0.1961, 0.1961, 0.

In [16]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()

    # Batch Normalization
    self.batch1 = nn.BatchNorm2d(3)

    # Capa de Dropout
    self.dropout1 = nn.Dropout(0.2)
    self.dropout2 = nn.Dropout(0.4)

    # Convoluciones
    self.conv1 = nn.Conv2d(3, 12, 5)  # Salida de 28
    self.conv2 = nn.Conv2d(12, 24, 3)  # Salida 12
    self.conv3 = nn.Conv2d(24, 30, 3)  # Salida 4

    # Capa Lineal
    self.fc1 = nn.Linear(30 * 2 * 2, 100)
    self.fc2 = nn.Linear(100, 80)
    self.fc3 = nn.Linear(80, 40)
    self.fc4 = nn.Linear(40, 10)

  def forward(self, x):
    # Batch
    x = self.batch1(x)
    x = self.conv1(x)
    x = torch.tanh(x)
    x = F.max_pool2d(x, (2,2))

    x = self.conv2(x)
    x = torch.tanh(x)
    x = F.max_pool2d(x, (2,2))

    x = self.dropout1(x)
    x = self.conv3(x)
    x = torch.tanh(x)
    x = F.max_pool2d(x, (2,2))

    x = x.view(-1, 30 * 2 * 2)
    x = self.fc1(x)
    x = torch.tanh(x)

    x = self.dropout2(x)
    x = self.fc2(x)
    x = torch.tanh(x)

    x = self.fc3(x)
    x = torch.tanh(x)

    x = self.fc4(x)
    return x

net = Net()
net.to(device)
print(net)

Net(
  (batch1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (dropout2): Dropout(p=0.4, inplace=False)
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(24, 30, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=80, bias=True)
  (fc3): Linear(in_features=80, out_features=40, bias=True)
  (fc4): Linear(in_features=40, out_features=10, bias=True)
)


In [17]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [19]:
from timeit import default_timer as timer
# Inicializar la pérdida = loss
# Coger las imágenes de entrenamiento
# Inicializar el gradiente
# Forward + Backward + optimize
start_total = timer()
for epoch in range(5):
  running_loss = 0.0
  start = timer()

  print(f"Vamos por la epoch: {epoch}")
  for i, data in enumerate(trainloader, 0):
    
    inputs, labels = data

    optimizer.zero_grad()

    # Pase para delante
    output = net(inputs.to(device))
    loss = criterion(output, labels.to(device))
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    if i % 2000 == 1999:
      print(f"Vamos por la iteración {i + 1} la pérdida es {loss:.2f} y la total es {running_loss:.2f}")
      end = timer()
      print(f"Ha pasado {(end - start):.2f} segundos")
      start = timer()

end_total = timer()
print(f"Finish Training, el modelo se ha entrenado en {(end_total - start_total):.2f} segundos")

Vamos por la epoch: 0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Vamos por la iteración 2000 la pérdida es 2.25 y la total es 3877.39
Ha pasado 8.76 segundos
Vamos por la iteración 4000 la pérdida es 1.93 y la total es 7337.60
Ha pasado 8.75 segundos
Vamos por la iteración 6000 la pérdida es 1.38 y la total es 10671.91
Ha pasado 8.64 segundos
Vamos por la iteración 8000 la pérdida es 2.26 y la total es 13828.07
Ha pasado 8.79 segundos
Vamos por la iteración 10000 la pérdida es 1.42 y la total es 16896.55
Ha pasado 8.67 segundos
Vamos por la epoch: 1
Vamos por la iteración 2000 la pérdida es 0.95 y la total es 2985.40
Ha pasado 8.63 segundos
Vamos por la iteración 4000 la pérdida es 1.07 y la total es 5948.53
Ha pasado 8.78 segundos
Vamos por la iteración 6000 la pérdida es 1.53 y la total es 8856.56
Ha pasado 8.59 segundos
Vamos por la iteración 8000 la pérdida es 1.14 y la total es 11717.15
Ha pasado 8.68 segundos
Vamos por la iteración 10000 la pérdida es 2.34 y la total es 14545.35
Ha pasado 8.85 segundos
Vamos por la epoch: 2
Vamos por la iterac

In [12]:
imagen = iter(testloader)

In [13]:
input, label = imagen.next()

In [109]:
input.to(device)
label.to(device)

tensor([1, 0, 4, 7, 0], device='cuda:0')

In [93]:
label

tensor([5, 4, 9, 9, 0])

In [15]:
net(input.to(device))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


tensor([[-0.0075, -0.0672, -0.1388, -0.0525,  0.0232, -0.0518, -0.0743,  0.1111,
          0.0402,  0.0131],
        [ 0.0077, -0.0677, -0.1270, -0.0521,  0.0270, -0.0501, -0.0856,  0.1165,
          0.0623,  0.0202],
        [-0.0239, -0.0642, -0.1442, -0.0386,  0.0302, -0.0441, -0.0891,  0.1254,
          0.0261,  0.0191],
        [-0.0147, -0.0733, -0.1250, -0.0582,  0.0240, -0.0397, -0.0726,  0.1127,
          0.0370,  0.0160],
        [ 0.0007, -0.0702, -0.1324, -0.0623,  0.0263, -0.0456, -0.0883,  0.1109,
          0.0474,  0.0095]], device='cuda:0', grad_fn=<AddmmBackward>)

In [95]:
torch.max(net(input), 1)

torch.return_types.max(values=tensor([2.2845, 4.0265, 8.0301, 7.8146, 3.1987], grad_fn=<MaxBackward0>), indices=tensor([3, 7, 9, 9, 8]))

In [69]:
label

tensor([9, 2, 0, 5, 5])

In [20]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images.to(device))
        # torch.max nos dice cuál es la clase dominante (la de mayor probabildad del vector de salida)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Accuracy of the network on the 10000 test images: 55 %
